# Stellargraph example: Graph Convolutional Network (GCN)

We demonstrated the use of GCN for node attribute inference on the CORA paper citation dataset.

**References**

[1] Semi-Supervised Classification with Graph Convolutional Networks, T. N. Kipf and M. Welling, ICLR 2017


Copyright 2010-2019 Commonwealth Scientific and Industrial Research Organisation (CSIRO).

All Rights Reserved.

Import NetworkX and stellar:

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, feature_extraction, model_selection

### Loading the CORA network

### Dataset


The dataset is the citation network Cora.

It can be downloaded by clicking [here](https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz)

The following is the description of the dataset from the publisher,

> The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words. The README file in the dataset provides more details. 

For this demo, we ignore the word vectors associated with each paper. We are only interested in the network structure and the **subject** attribute of each paper.

Download and unzip the cora.tgz file to a location on your computer. 

We assume that the dataset is stored in the directory

`../data/cora/`

where the files `cora.cites` and `cora.content` can be located.

We are going to load the data into a networkx object.

In [ ]:
data_dir = os.path.expanduser("../data/cora")

In [ ]:
cora_location = os.path.expanduser(os.path.join(data_dir, "cora.cites"))
g_nx = nx.read_edgelist(path=cora_location)

# load the node attribute data
cora_data_location = os.path.expanduser(os.path.join(data_dir, "cora.content"))
node_attr = pd.read_csv(cora_data_location, sep='\t', header=None)
values = { str(row.tolist()[0]): row.tolist()[-1] for _, row in node_attr.iterrows()}
nx.set_node_attributes(g_nx, values, 'subject')

# Select the largest connected component. For clarity we ignore isolated
# nodes and subgraphs; having these in the data does not prevent the
# algorithm from running and producing valid results.
g_nx_ccs = (g_nx.subgraph(c).copy() for c in nx.connected_components(g_nx))
g_nx = max(g_nx_ccs, key=len)
print("Largest subgraph statistics: {} nodes, {} edges".format(
    g_nx.number_of_nodes(), g_nx.number_of_edges()))

Load the features and subject for the nodes

In [ ]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_table(os.path.join(data_dir, "cora.content"), header=None, names=column_names)

In [ ]:
node_data.index = node_data.index.map(str)
node_data = node_data[node_data.index.isin(list(g_nx.nodes()))]

We aim to train a graph-ML model that will predict the "subject" attribute on the nodes. These subjects are one of 7 categories:

In [ ]:
set(node_data["subject"])

### Splitting the data

For machine learning we want to take a subset of the nodes for training, and use the rest for validation and testing. We'll use scikit-learn again to do this.

Here we're taking 140 node labels for training, 500 for validation, and the rest for testing.

In [ ]:
train_data, test_data = model_selection.train_test_split(node_data, train_size=140, test_size=None, stratify=node_data['subject'], random_state=42)
val_data, test_data = model_selection.train_test_split(test_data, train_size=500, test_size=None, stratify=test_data['subject'])

Note using stratified sampling gives the following counts:

In [ ]:
from collections import Counter
Counter(train_data['subject'])

The training set has class imbalance that might need to be compensated, e.g., via using a weighted cross-entropy loss in model training, with class weights inversely proportional to class support. However, we will ignore the class imbalance in this example, for simplicity.

### Converting to numeric arrays

For our categorical target, we will use one-hot vectors that will be fed into a soft-max Keras layer during training. To do this conversion ...

In [ ]:
target_encoding = feature_extraction.DictVectorizer(sparse=False)

train_targets = target_encoding.fit_transform(train_data[["subject"]].to_dict('records'))
val_targets = target_encoding.transform(val_data[["subject"]].to_dict('records'))
test_targets = target_encoding.transform(test_data[["subject"]].to_dict('records'))

We now do the same for the node attributes we want to use to predict the subject. These are the feature vectors that the Keras model will use as input. The CORA dataset contains attributes 'w_x' that correspond to words found in that publication. If a word occurs more than once in a publication the relevant attribute will be set to one, otherwise it will be zero.

In [ ]:
node_features = node_data[feature_names]

In [ ]:
node_features.shape

In [ ]:
node_features.head(3)

## Creating the GCN model in Keras

Now create a StellarGraph object from the NetworkX graph and the node features and targets. It is StellarGraph objects that we use in this library to perform machine learning tasks on.

In [ ]:
G = sg.StellarGraph(g_nx, node_features=node_features)

In [ ]:
print(G.info())

To feed data from the graph to the Keras model we need a generator. Since GCN is a full-batch model, we use the `FullBatchNodeGenerator` class to feed node features and graph adjacency matrix to the model.

In [ ]:
generator = FullBatchNodeGenerator(G)

For training we map only the training nodes returned from our splitter and the target values.

In [ ]:
train_gen = generator.flow(train_data.index, train_targets)

Now we can specify our machine learning model, we need a few more parameters for this:

 * the `layer_sizes` is a list of hidden feature sizes of each layer in the model. In this example we use two GCN layers with 32-dimensional hidden node features at the first layer and 7 for the second layer (output layer).
 * `activations` is a list of activations applied to each layer's output
 * Arguments such as `bias` and `dropout` are internal parameters of the model, execute `?GCN` for details. 

In [ ]:
gcn = GCN(
    layer_sizes=[32, 7],
    generator=generator,
    bias=True,
    dropout=0.5,
    activations=["elu","softmax"],
)

In [ ]:
# Expose the input and output tensors of the GCN model for node prediction, 
# via GCN.node_model() method:
x_inp, x_out = gcn.node_model()

Note that `x_inp` is a list of two input tensors, one for node features, the other for graph adjacency matrix

In [ ]:
x_inp

Now we add the final layer to predict the 7 categories using Keras Dense layer with softmax activation.

In [ ]:
# Snap the final estimator layer to x_out
#predictions = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)
predictions = x_out

In [ ]:
predictions.shape

### Training the model

Now let's create the actual Keras model with the input tensors `x_inp` and output tensors being the predictions `predictions` from the final dense layer

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)
model.compile(
    optimizer=optimizers.Adam(lr=0.005),
    loss=losses.categorical_crossentropy,
    weighted_metrics=["acc"],
)

Train the model, keeping track of its loss and accuracy on the training set, and its generalisation performance on the validation set (we need to create another generator over the validation data for this)

In [ ]:
val_gen = generator.flow(val_data.index, val_targets)

Create callbacks for early stopping (if validation accuracy stops improving) and best model checkpoint saving:

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# patience is the number of epochs to wait before early stopping in case of no further improvement
es_callback = EarlyStopping(monitor="val_acc", patience=50, restore_best_weights=True)  

Train the model

In [ ]:
history = model.fit_generator(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    verbose=1,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
    callbacks=[es_callback],
)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# def remove_prefix(text, prefix):
#     return text[text.startswith(prefix) and len(prefix):]

# def plot_history(history):
#     metrics = sorted(set([remove_prefix(m, "val_") for m in list(history.history.keys())]))
#     for m in metrics:
#         # summarize history for metric m
#         plt.plot(history.history[m])
#         plt.plot(history.history['val_' + m])
#         plt.title(m)
#         plt.ylabel(m)
#         plt.xlabel('epoch')
#         plt.legend(['train', 'validation'], loc='right')
#         plt.show()

In [ ]:
sg.utils.plot_history(history)

Evaluate the best model on the test set

In [ ]:
test_gen = generator.flow(test_data.index, test_targets)

In [ ]:
test_metrics = model.evaluate_generator(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

### Making predictions with the model

Now let's get the predictions themselves for all nodes:

In [ ]:
all_nodes = G.nodes()
all_gen = generator.flow(all_nodes)
all_predictions = model.predict_generator(all_gen).squeeze()

In [ ]:
all_predictions.shape

These predictions will be the output of the softmax layer, so to get final categories we'll use the `inverse_transform` method of our target attribute specifcation to turn these values back to the original categories

In [ ]:
node_predictions = target_encoding.inverse_transform(all_predictions)

Let's have a look at a few (note that generator orders nodes according to the order in G.nodes(), so the predictions are ordered like that as well. We thus need to index results as `G.nodes()`

In [ ]:
#node_data.reindex(G.nodes())
results = pd.DataFrame(node_predictions, 
                       index=G.nodes()).idxmax(axis=1)
df = pd.DataFrame({"Predicted": results, "True": node_data['subject']})
df.head(20)

## Node embeddings
Evaluate node embeddings as activations of the output of GCN layer stack, and visualise them, coloring nodes by their subject label.

The GCN embeddings are the output of one of the GCN layers in the model; for example, this can be the `x_out` variable if we use the output of the last layer. 

Let's create a new model with the same inputs as we used previously `x_inp` but now the output is the embeddings rather than the predicted class. Additionally note that the weights trained previously are kept in the new model.

In [ ]:
# Let's have a look at the layers in the model.
model.layers

We can take the output activations from one of the GCN layers, e.g., the first one in a two-layer model as the embedding vectors. Alternatively, we can use the 7-dimensional output of the second GCN layer as the embedding vectors.

In [ ]:
#
# Use the output of the first GCN layer
embedding_model = Model(inputs=x_inp, outputs=model.layers[6].output)
# or
# Use the output of the second GCN layer
#embedding_model = Model(inputs=x_inp, outputs=x_out)

In [ ]:
emb = embedding_model.predict_generator(all_gen).squeeze()
emb.shape

Project the embeddings to 2d using either TSNE or PCA transform, and visualise, coloring nodes by their subject label

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np

Note that the generator orders nodes according to the order in G.nodes(), so we need to re-index node_data

In [ ]:
X = emb
y = np.argmax(target_encoding.transform(node_data.reindex(G.nodes())[["subject"]].to_dict('records')), axis=1)

In [ ]:
if X.shape[1] > 2:
    transform = TSNE #PCA 

    trans = transform(n_components=2)
    emb_transformed = pd.DataFrame(trans.fit_transform(X), 
                                   index=list(G.nodes()))
    emb_transformed['label'] = y
else:
    emb_transformed = pd.DataFrame(X, index=list(G.nodes()))
    emb_transformed = emb_transformed.rename(columns = {'0':0, '1':1})
    emb_transformed['label'] = y

In [ ]:
alpha = 0.7

fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(emb_transformed[0], emb_transformed[1], 
           c=emb_transformed['label'].astype("category"), 
            cmap="jet", alpha=alpha)
ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title('{} visualization of GCN embeddings for cora dataset'.format(transform.__name__))
plt.show()
#fig.savefig("GCN_with_features.png")